# Word Classification

This notebook provides an introduction on using NLTK and Scikit-Learn for performing Word Classification

## Initialize NTLK

Download some of the resources that NLTK needs

In [ ]:
import nltk
nltk.download('book')
nltk.download('opinion_lexicon')

## Import the additional modules

The `random` module is loaded to do some randomization on the raw data. The seed is set so that repeated runs of the notebook is replicable. 

While NLTK also contains implementation of machine learning algorithms, Scikit-Learn provides general purpose machine learning implementations. Learning to use its modules meaning learning to use them on data outside NLP

In [ ]:
import random
random.seed(0)

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import (
    classification_report,
    roc_auc_score,
    confusion_matrix,
)

## Word Classification

In this part a proposition is to be verified. To word structures affect the meaning of the words. That is so say, is there a common pattern on negative words and positive words? 

### Loading the data

NLTK provides the `opinion_lexicon` that list words that are negative and positive in nature. The words can be loaded as shown below.

In [ ]:
negatives = nltk.corpus.opinion_lexicon.negative()
negatives

In [ ]:
len(negatives)

In [ ]:
positives = nltk.corpus.opinion_lexicon.positive()
positives

In [ ]:
len(positives)

### Balancing the Dataset

From the number of data, there are more negative words than positive words. This will bias most machine learning algorithms. As a simple solution, the number of negative words will be reduced to balance the number.

The negatives are shuffled before reducing the number to make sure there is a sample for every starting letter of the alphabet. It is converted to a list for an inplace shuffle.

In [ ]:
negatives = list(negatives)
random.shuffle(negatives)

In [ ]:
negatives = negatives[:len(positives)]

In [ ]:
words = negatives + list(positives)
labels = [0] * len(negatives) + [1] * len(positives)

len(words), len(labels)

### Feature Engineering

Given that some English affixes can change the meaning of words, 3-character suffixes and prefixes are used as an initial feature in determining the word sentiment.

In [ ]:
def word_features(word):
    return {
        '3-suffix': word[-3:],
        '3-prefix': word[:3]
    }

In [ ]:
word_features = [
    word_features(w)
    for w in words
]

In [ ]:
word_features[:10]

### Splitting the Dataset

The dataset will be split into 3 different sets, the training set, the validation set, and the testing set.

*   Train (70%): Used for training the machine learning model
*   Validation (10%): Used for evaluation of pipeline changes such as feature engineering and model hyperparameters
*   Test (20%): Used to evaluate the whole pipeline.

The scikit-learn `train_test_split` only splits in two sets. Thus, it will be used twice. The `strafity` options makes sures that the split will consider the balance of the labels while `random_state` sets the seed to allow replicability

In [ ]:
train_features, test_features, train_labels, test_labels = (
    train_test_split(word_features, labels, test_size=0.2, stratify=labels, random_state=0)
)

train_features, val_features, train_labels, val_labels = (
    train_test_split(train_features, train_labels, test_size=1/8, stratify=train_labels, random_state=0)
)

len(train_features), len(test_features), len(val_features)

### Dictionary to Vectors

Machine learning algorithms work with vectors rather than dictionary of features. Scikit-Learn's `DictVectorizer` converts the dictionary of features into a vector.

Take note that on the training the `fit_transform` method is used. This means that only the features in the training will be considerd in vectorization. The validation and test only used the `transform` method.

The `DictVectorizer` will convert any non numeric data into one-hot encoding. This means that each value will be created as a flag.

In [ ]:
dv = DictVectorizer(sparse=False)
train_vectors = dv.fit_transform(train_features)
val_vectors = dv.transform(val_features)
test_vectors = dv.transform(test_features)

In [ ]:
dv.feature_names_[:10], dv.feature_names_[-10:]

### Naive Bayes

The `BernoulliNB` is an implementation of Naive Bayes that is designed for boolean/flags. Note that the `fit` method is only called on the train dataset

In [ ]:
nb = BernoulliNB()
nb.fit(train_vectors, train_labels)

In [ ]:
train_predict = nb.predict(train_vectors)
val_predict = nb.predict(val_vectors)
test_predict =  nb.predict(test_vectors)

### Evaluation

The different sets will be evaluated to show how the performance degrades from the training data and the out of sample data. The following metrics are computed for each set.

*   ROC AUC
*   Precision
*   Recall
*   F1-Score
*   Accuracy
*   Confusion Matrix

#### Training Performance

In [ ]:
roc_auc_score(train_labels, train_predict)

In [ ]:
print(classification_report(train_labels, train_predict))

In [ ]:
confusion_matrix(train_labels, train_predict)

#### Validation Performance

In [ ]:
roc_auc_score(val_labels, val_predict)

In [ ]:
print(classification_report(val_labels, val_predict))

In [ ]:
confusion_matrix(val_labels, val_predict)

#### Test Performance

In [ ]:
roc_auc_score(test_labels, test_predict)

In [ ]:
print(classification_report(test_labels, test_predict))

In [ ]:
confusion_matrix(test_labels, test_predict)

### Interpretability

One of the advantages of Naive Bayes is its simplicity. The simplicity allows the deicisions taken by the machine learning model to be analyzed.

On the first part, the most common features for both labels are verified. Then the features that are biased on the labels are checked on the second part

#### Highest probabilities

By checking the features with the highest probability at each side, the common features at both labels can be examined.

For example, `ing`, `ble`, `ess`, and `ion` are in the top of both labels. This makes sense since these are usually suffixes for descriptive words,

In [ ]:
best_neg = nb.feature_log_prob_[0].argsort()
best_pos = nb.feature_log_prob_[1].argsort()

In [ ]:
[dv.feature_names_[idx] for idx in best_neg[-10:][::-1]]

In [ ]:
[dv.feature_names_[idx] for idx in best_pos[-10:][::-1]]

#### Biased Features

By getting the difference between probabilities, the features that are biased to the labels are be determined.

For example, `dis`, `mis`, `ina`, `mal`, and `ant` are pretty common prefixes on the negative labels. This makes sense given that most of these prefixes are negating prefixes, creating a negative opinion.

In [ ]:
diff_pos_neg = (nb.feature_log_prob_[1] - nb.feature_log_prob_[0]).argsort()

In [ ]:
[dv.feature_names_[idx] for idx in diff_pos_neg[:10]]

In [ ]:
[dv.feature_names_[idx] for idx in diff_pos_neg[-10:][::-1]]